In [0]:
from pyspark.sql import SparkSession
from pyspark import SparkFiles



In [0]:
# Inicializar la sesión de Spark
spark = SparkSession.builder.appName("merge_csv_files").getOrCreate()

In [0]:
# Ruta del contenedor en Azure Databricks
container_path = "/mnt/source/"

In [0]:
# Obtener la lista de archivos en el contenedor
files = [file.path for file in dbutils.fs.ls(container_path) if file.path.endswith(".csv")]

In [0]:
# Verificar si hay archivos para procesar
if not files:
    print("No hay archivos CSV para procesar en el contenedor.")
else:
    # Leer y combinar archivos CSV
    combined_df = None

    for file in files:
        df = spark.read.csv(file, header=True, inferSchema=True)

        if combined_df is None:
            combined_df = df
        else:
            combined_df = combined_df.union(df)

    # Reducir el número de particiones a 1
    combined_df = combined_df.coalesce(1)

    # Guardar el DataFrame combinado en un solo archivo CSV
    combined_output_path = "/mnt/bronze/Eco_Bici_2021_2023.csv"
    df_sin_nulos = combined_df.dropna(how="any")
    df_sin_nulos.write.parquet(combined_output_path, mode="overwrite")

    print(f"Archivos combinados exitosamente. Resultado guardado en: {combined_output_path}")

Archivos combinados exitosamente. Resultado guardado en: /mnt/bronze/Eco_Bici_2021_2023.csv
